# Observing the behavior of a DPLL and a CDCL SAT solver

Let's see how we can observe how a DPLL solver and a CDCL solver work, and the difference between them.

In [1]:
from cdcl_solver import CDCLSolver
from dpll_solver import DPLLSolver

## Load a formula

Let's load a CNF formula for the solvers to run on.

In [2]:
cnf_path = "example2.cnf"

The CNF formula should be in *DIMACS CNF format*. This is a plain text file that consists of the following:
- It may (but does not have to) start with some comment lines, that each start with `c `, and these are ignored by the solver.
- Then there is a header line of the form `p cnf NUM_VARS NUM_CLAUSES`, where `NUM_VARS` is the number of variables occurring in the formula (and these should be numbered 1, 2, ..., `NUM_VARS`), and where `NUM_CLAUSES` is the number of clauses in the formula.
- The following lines then encode a clause each. Each clause is encoded by writing the literals occurring in it, followed by a `0`, all separated by single spaces. Literals are encoded by integers: a positive integer `i` encodes the positive literal $x_i$, and a negative integer `-i` encodes the negative literal $\neg x_i$.
  * For example, the clause $(x_1 \vee \neg x_2 \vee x_3)$ is encoded as `1 -2 3 0`.

Let's inspect the example formula that we loaded to see what DIMACS CNF format looks like.

In [3]:
with open(cnf_path, "r", encoding="utf-8") as source_file:
    for line in source_file.readlines():
        print(line, end="")

c This is an example CNF formula that nicely shows the difference
c in behavior between a DPLL and a CDCL solver,
c when using the ORDERED decision heuristic.
c
c It has 8 variables and 9 clauses.
c
p cnf 8 9
-3 -4 -5 6 0
-1 -6 7 0
-6 -8 0
-1 -7 8 0
-1 -3 6 0
-1 -4 6 0
-1 -5 6 0
-1 3 4 5 6 -7 0
-1 -2 3 4 5 6 7 0


## Running the DPLL solver

Let's now call the DPLL solver on our example formula.

Our implementation of the DPLL solver uses the `"ORDERED"` branching heuristic. This means that whenever the solver cannot propagate any further, it takes the variable with the smallest index among the still unassigned variables, and branches by setting this variable to True.

When we run the algorithm (with `verbose=True`), it gives us a trace of what happened along the way, and a final answer (`SAT` with a satisfying assignment, or `UNSAT`).

In [4]:
solver = DPLLSolver(
    verbose=True,
)
solver.solve(cnf_path)

> Decision:    1    (level: 1)
  Current assignment: [1]
> Decision:    2    (level: 2)
  Current assignment: [1, 2]
> Decision:    3    (level: 3)
  Current assignment: [1, 2, 3]
* UP:          6    (level: 3)
  Current assignment: [1, 2, 3, 6]
* UP:          -8   (level: 3)
  Current assignment: [1, 2, 3, 6, -8]
* UP:          7    (level: 3)
  Current assignment: [1, 2, 3, 6, -8, 7]
= CONFLICT!
< Backtracking to level 2
* Propagated:  -3   (level: 2)
  Current assignment: [1, 2, -3]
> Decision:    4    (level: 3)
  Current assignment: [1, 2, -3, 4]
* UP:          6    (level: 3)
  Current assignment: [1, 2, -3, 4, 6]
* UP:          -8   (level: 3)
  Current assignment: [1, 2, -3, 4, 6, -8]
* UP:          7    (level: 3)
  Current assignment: [1, 2, -3, 4, 6, -8, 7]
= CONFLICT!
< Backtracking to level 2
* Propagated:  -4   (level: 2)
  Current assignment: [1, 2, -3, -4]
> Decision:    5    (level: 3)
  Current assignment: [1, 2, -3, -4, 5]
* UP:          6    (level: 3)
  Current ass

So what does the different parts of this trace mean?
- The final answer, at the very bottom, is `SAT`, meaning that the formula is satisfiable. The assignment directly before it is a satisfying assignment.
- Throughout the search, the algorithm keeps track of a current partial assignment.
- Whenever the current assignment allows the algorithm to deduce further literals using unit propagation, this is indicated with `* UP`.
- If no further propagation is possible, the algorithm branches by setting a literal to true (indicated with `> Decision`).
- The *decision level* is the current number of decisions made that the algorithm can still backtrack on. This is indicated with `level` at each line.
- Whenever the current assignment falsifies the formula (`CONFLICT`), the algorithm backtracks by undoing the most recent decision, and concluding that the opposite of this decision should hold (`Propagated`).

## Running the CDCL solver

Let's now call the CDCL solver on our example formula.

Let's start by using the same branching heuristic (`"ORDERED"`), and by not using any restarts.

In [5]:
solver = CDCLSolver(
    branching="ORDERED",
    restart=None,
    verbose=True,
)
solver.solve(cnf_path)

> Decision:    1    (level: 1)
  Current assignment: [1]
> Decision:    2    (level: 2)
  Current assignment: [1, 2]
> Decision:    3    (level: 3)
  Current assignment: [1, 2, 3]
* UP:          6    (level: 3)
  Current assignment: [1, 2, 3, 6]
* UP:          -8   (level: 3)
  Current assignment: [1, 2, 3, 6, -8]
* UP:          7    (level: 3)
  Current assignment: [1, 2, 3, 6, -8, 7]
= CONFLICT! Learned clause: [-1, -6] 
< Backjumping to level 1
* Propagated:  -6   (level: 1)
  Current assignment: [1, -6]
* UP:          -3   (level: 1)
  Current assignment: [1, -6, -3]
* UP:          -4   (level: 1)
  Current assignment: [1, -6, -3, -4]
* UP:          -5   (level: 1)
  Current assignment: [1, -6, -3, -4, -5]
* UP:          -7   (level: 1)
  Current assignment: [1, -6, -3, -4, -5, -7]
* UP:          -2   (level: 1)
  Current assignment: [1, -6, -3, -4, -5, -7, -2]
> Decision:    8    (level: 2)
  Current assignment: [1, -6, -3, -4, -5, -7, -2, 8]
SAT


The difference with the DPLL algorithm is that when a conflict is reached, the algorithm doesn't necessarily undo the most recent decision. Instead, it analyzes the conflict, learns a new clause (`Learned clause`), and uses this clause to *backjump* as much as possible using this learned clause.

### Different branching and restart strategies

Our implementation of the CDCL solver also supports different heuristics for branching and for restarts.

The two branching heuristics (`"VSIDS"` and `"MINISAT"`) both keep track of how often variables/literals occur in the formula, and how often variables/literals are involved in a conflict, and decide which variable to set based on this information (in slightly different ways).

In [6]:
solver = CDCLSolver(
    branching="VSIDS",
    restart=None,
    verbose=True,
)
solver.solve(cnf_path)

> Decision:    -1   (level: 1)
  Current assignment: [-1]
> Decision:    6    (level: 2)
  Current assignment: [-1, 6]
* UP:          -8   (level: 2)
  Current assignment: [-1, 6, -8]
> Decision:    -5   (level: 3)
  Current assignment: [-1, 6, -8, -5]
> Decision:    4    (level: 4)
  Current assignment: [-1, 6, -8, -5, 4]
> Decision:    -7   (level: 5)
  Current assignment: [-1, 6, -8, -5, 4, -7]
> Decision:    -3   (level: 6)
  Current assignment: [-1, 6, -8, -5, 4, -7, -3]
> Decision:    -2   (level: 7)
  Current assignment: [-1, 6, -8, -5, 4, -7, -3, -2]
SAT


In [7]:
solver = CDCLSolver(
    branching="MINISAT",
    restart=None,
    verbose=True,
)
solver.solve(cnf_path)

> Decision:    -6   (level: 1)
  Current assignment: [-6]
> Decision:    -1   (level: 2)
  Current assignment: [-6, -1]
> Decision:    -7   (level: 3)
  Current assignment: [-6, -1, -7]
> Decision:    -4   (level: 4)
  Current assignment: [-6, -1, -7, -4]
> Decision:    -5   (level: 5)
  Current assignment: [-6, -1, -7, -4, -5]
> Decision:    -3   (level: 6)
  Current assignment: [-6, -1, -7, -4, -5, -3]
> Decision:    -8   (level: 7)
  Current assignment: [-6, -1, -7, -4, -5, -3, -8]
> Decision:    -2   (level: 8)
  Current assignment: [-6, -1, -7, -4, -5, -3, -8, -2]
SAT


The implementation of the CDCL solver also supports two different restart strategies (`"GEOMETRIC"` and `"LUBY"`). These both tell the solver to restart the search (while keeping learned clauses) after a certain amount of steps. The number of steps between restarts increases (roughly speaking).

For small examples, this doesn't make a difference, as the number of steps between restarts starts at 512, and increases from there.

In [8]:
solver = CDCLSolver(
    branching="ORDERED",
    restart="LUBY",
    verbose=True,
)
solver.solve(cnf_path)

> Decision:    1    (level: 1)
  Current assignment: [1]
> Decision:    2    (level: 2)
  Current assignment: [1, 2]
> Decision:    3    (level: 3)
  Current assignment: [1, 2, 3]
* UP:          6    (level: 3)
  Current assignment: [1, 2, 3, 6]
* UP:          -8   (level: 3)
  Current assignment: [1, 2, 3, 6, -8]
* UP:          7    (level: 3)
  Current assignment: [1, 2, 3, 6, -8, 7]
= CONFLICT! Learned clause: [-1, -6] 
< Backjumping to level 1
* Propagated:  -6   (level: 1)
  Current assignment: [1, -6]
* UP:          -3   (level: 1)
  Current assignment: [1, -6, -3]
* UP:          -4   (level: 1)
  Current assignment: [1, -6, -3, -4]
* UP:          -5   (level: 1)
  Current assignment: [1, -6, -3, -4, -5]
* UP:          -7   (level: 1)
  Current assignment: [1, -6, -3, -4, -5, -7]
* UP:          -2   (level: 1)
  Current assignment: [1, -6, -3, -4, -5, -7, -2]
> Decision:    8    (level: 2)
  Current assignment: [1, -6, -3, -4, -5, -7, -2, 8]
SAT
